In [4]:
df_dict = pd.read_excel("Excel donnés qualité UGB.xlsx", sheet_name=None, header=3)

for name, data in df_dict.items():
    print(f"{name}: {data.shape}, colonnes: {list(data.columns)}")


Entree 1: (16, 21), colonnes: ['Mois', 'Data', 'Heure echantillon ', 'Debit entree (m3/jour)', 'Temperature (ºC)', 'pH', 'CE (µSm/cm)', 'Redox', 'DBO5 (mg/L)', 'DCO (mg/L)', 'MeS (mg/L)', 'MVS (%)', 'Nitrates (mgNO3-/l)', 'Ammonium (mgNH4-/l) ', 'Azot total (mgN/l)', 'Phosphates (mgPO4-/l)', 'Coliformes fécaux (CFU/100ml)', 'Oeufs helmint', 'Huiles et graisses', 'Opération', 'Observation']
Entree FV1: (16, 21), colonnes: ['Mois', 'Data', 'Heure echantillon /observations', 'Debit entree (m3/jour)', 'Temperature (ºC)', 'pH', 'CE (µsm/cm)', 'Redox', 'DBO5 (mg/L)', 'DCO (mg/L)', 'MeS (mg/L)', 'MVS (%)', 'Nitrates (mgNO3-/l)', 'Ammonium (mgNH4-/l) ', 'Azote total (mgN/l)', 'Phosphates (mgPO4-/l)', 'Coliformes fécaux (CFU/100ml)', 'Oeufs helmint', 'Huiles et graisses', 'Opération', 'Observation']
SFV1a: (16, 21), colonnes: ['Mois', 'Data', 'Heure echantillon /observations', 'Charge hydraulique (cm/jour)', 'Temperature (ºC)', 'pH', 'CE (µsm/cm)', 'Redox', 'DBO5 (mg/L)', 'DCO (mg/L)', 'MeS (mg

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

class UGBDataProcessor:
    """
    Classe pour le prétraitement des données de qualité d'eau UGB
    """
    
    def __init__(self):
        # Mapping pour uniformiser les noms de colonnes
        self.column_mapping = {
            # Colonnes temporelles
            'Mois': 'Mois',
            'Data': 'Date',
            'Heure echantillon ': 'Heure_Echantillon',
            'Heure echantillon /observations': 'Heure_Echantillon',
            
            # Paramètres hydrauliques
            'Debit entree (m3/jour)': 'Debit_Entree_m3_jour',
            'Charge hydraulique (cm/jour)': 'Charge_Hydraulique_cm_jour',
            
            # Paramètres physico-chimiques
            'Temperature (ºC)': 'Temperature_C',
            'pH': 'pH',
            'CE (µSm/cm)': 'Conductivite_uS_cm',
            'CE (µsm/cm)': 'Conductivite_uS_cm',
            'Redox': 'Potentiel_Redox_mV',
            
            # Paramètres de pollution
            'DBO5 (mg/L)': 'DBO5_mg_L',
            'DCO (mg/L)': 'DCO_mg_L',
            'MeS (mg/L)': 'MES_mg_L',
            'MVS (%)': 'MVS_pct',
            
            # Paramètres azotés (TOUTES LES VARIANTES vers le même nom)
            'Nitrates (mgNO3-/l)': 'Nitrates_mg_L',
            'Ammonium (mgNH4-/l) ': 'Ammonium_mg_L',
            'Azot total (mgN/l)': 'Azote_Total_mg_L',      # Variante 1
            'Azote total (mgN/l)': 'Azote_Total_mg_L',     # Variante 2 
            'Azote toal (mgN/l)': 'Azote_Total_mg_L',      # Variante 3 (typo)
            
            # Paramètres phosphorés
            'Phosphates (mgPO4-/l)': 'Phosphates_mg_L',
            
            # Paramètres microbiologiques
            'Coliformes fécaux (CFU/100ml)': 'Coliformes_Fecaux_CFU_100ml',
            'Oeufs helmint': 'Oeufs_Helminthes',
            
            # Autres
            'Huiles et graisses': 'Huiles_Graisses',
            'Opération': 'Operation',
            'Observation': 'Observations'
        }
        
        # Définition des types de feuilles et leurs caractéristiques
        self.sheet_config = {
            'entree': {
                'sheets': ['Entree 1', 'Entree FV1', 'Entréé FV2'],
                'phase': 'Entree',
                'description': 'Eaux d\'entrée'
            },
            'sortie_fv': {
                'sheets': ['SFV1a', 'SFV1b', 'SFV1c', 'SFV2a', 'SFV2b'],
                'phase': 'Sortie',
                'type_filtre': 'Filtre_Vertical',
                'description': 'Sorties filtres verticaux'
            },
            'sortie_fh': {
                'sheets': ['SFHa', 'SFHb'],
                'phase': 'Sortie',
                'type_filtre': 'Filtre_Horizontal',
                'description': 'Sorties filtres horizontaux'
            },
            'performance': {
                'sheets': ['% él FV1', '% él FV2', '% él de la station'],
                'phase': 'Performance',
                'description': 'Données de performance'
            }
        }
    
    def clean_value(self, x):
        """
        Nettoie une valeur individuelle
        Gère spécifiquement : <2000, >500, "pas de donnée", etc.
        """
        if pd.isna(x):
            return np.nan
        
        if isinstance(x, str):
            x = x.strip()
            
            # Gérer les valeurs spéciales (valeurs manquantes)
            missing_indicators = ['pas de donnée', 'nd', 'n.d.', '-', 'absent', '', 'nan', 'null']
            if any(term in x.lower() for term in missing_indicators):
                return np.nan
            
            # Gérer les valeurs avec > (ex: ">500" devient 500)
            # Interprétation: valeur minimale possible
            if '>' in x:
                try:
                    numeric_part = x.replace('>', '').strip()
                    return float(numeric_part)
                except ValueError:
                    return np.nan
            
            # Gérer les valeurs avec < (ex: "<2000" devient 2000) 
            # Interprétation: valeur maximale possible (limite de détection)
            if '<' in x:
                try:
                    numeric_part = x.replace('<', '').strip()
                    return float(numeric_part)
                except ValueError:
                    return np.nan
            
            # Gérer d'autres formats possibles
            # Retirer les espaces et caractères parasites
            x = x.replace(',', '.').replace(' ', '')
            
            # Essayer de convertir en numérique
            try:
                return float(x)
            except ValueError:
                # Si ce n'est pas numérique, retourner tel quel (texte)
                return x if x else np.nan
        
        return x
    
    def standardize_columns(self, df, sheet_name):
        """
        Standardise les noms de colonnes d'un DataFrame
        """
        df_clean = df.copy()
        
        # Renommer les colonnes selon le mapping
        df_clean.rename(columns=self.column_mapping, inplace=True)
        
        # Ajouter les métadonnées
        df_clean['ID_Station'] = 'Sanar_Station'
        df_clean['Nom_Feuille'] = sheet_name
        
        # Déterminer le type et la phase selon la feuille
        for config_type, config in self.sheet_config.items():
            if sheet_name in config['sheets']:
                df_clean['Phase'] = config['phase']
                if 'type_filtre' in config:
                    df_clean['Type_Filtre'] = config['type_filtre']
                else:
                    df_clean['Type_Filtre'] = 'Non_Applicable'
                break
        
        # Identifier le filtre spécifique
        if 'FV1' in sheet_name:
            df_clean['ID_Filtre'] = 'FV1'
        elif 'FV2' in sheet_name:
            df_clean['ID_Filtre'] = 'FV2'
        elif 'FH' in sheet_name:
            df_clean['ID_Filtre'] = 'FH'
        else:
            df_clean['ID_Filtre'] = 'General'
        
        return df_clean
    
    def process_sheet(self, df, sheet_name):
        """
        Traite une feuille individuelle
        """
        print(f"Traitement de la feuille : {sheet_name}")
        
        # Standardiser les colonnes
        df_processed = self.standardize_columns(df, sheet_name)
        
        # Nettoyer les valeurs (sauf les métadonnées)
        metadata_cols = ['ID_Station', 'Nom_Feuille', 'Phase', 'Type_Filtre', 'ID_Filtre']
        
        for col in df_processed.columns:
            if col not in metadata_cols:
                df_processed[col] = df_processed[col].apply(self.clean_value)
        
        # Traiter la date
        if 'Date' in df_processed.columns:
            df_processed['Date'] = pd.to_datetime(df_processed['Date'], errors='coerce')
        
        return df_processed
    
    def create_unified_dataset(self, excel_file_path):
        """
        Crée un dataset unifié à partir du fichier Excel
        """
        print("=== DÉBUT DU PRÉTRAITEMENT UGB ===")
        
        # 1. Charger toutes les feuilles
        try:
            df_dict = pd.read_excel(excel_file_path, sheet_name=None)
            print(f"Nombre de feuilles chargées : {len(df_dict)}")
        except Exception as e:
            print(f"Erreur lors du chargement : {e}")
            return None
        
        # 2. Traiter chaque feuille (exclure les feuilles de performance pour l'instant)
        processed_sheets = []
        performance_sheets = []
        
        for sheet_name, data in df_dict.items():
            if sheet_name.startswith('% él'):
                # Traiter séparément les feuilles de performance
                performance_sheets.append((sheet_name, data))
                continue
            
            try:
                processed_df = self.process_sheet(data, sheet_name)
                processed_sheets.append(processed_df)
                print(f"  ✓ {sheet_name}: {len(processed_df)} lignes")
            except Exception as e:
                print(f"  ✗ Erreur avec {sheet_name}: {e}")
        
        # 3. Combiner toutes les feuilles
        if processed_sheets:
            df_unified = pd.concat(processed_sheets, ignore_index=True, sort=False)
            print(f"\nDataset unifié créé : {len(df_unified)} lignes, {len(df_unified.columns)} colonnes")
        else:
            print("Aucune feuille traitée avec succès")
            return None
        
        # 4. Analyser les valeurs manquantes
        missing_analysis = self.analyze_missing_values(df_unified)
        
        # 5. Gérer les valeurs manquantes 
        # CHOISIR UNE SEULE STRATÉGIE selon vos besoins :
        
        # Option 1 - HYBRID (Recommandée - approche intelligente)
        df_unified = self.handle_missing_values(df_unified, strategy='hybrid')
        
        # Option 2 - CONSERVATIVE (décommenter si vous voulez garder toutes les données)
        # df_unified = self.handle_missing_values(df_unified, strategy='conservative')
        
        # Option 3 - AGGRESSIVE (décommenter si vous voulez supprimer les colonnes très incomplètes)
        # df_unified = self.handle_missing_values(df_unified, strategy='aggressive')
        
        # Option 4 - INTERPOLATE (décommenter si vous voulez interpoler les séries temporelles)  
        # df_unified = self.handle_missing_values(df_unified, strategy='interpolate')
        
        # 6. Post-traitement final
        df_unified = self.post_process(df_unified)
        
        return df_unified, performance_sheets, missing_analysis
    
    def analyze_missing_values(self, df):
        """
        Analyse détaillée des valeurs manquantes
        """
        print("\n=== ANALYSE DES VALEURS MANQUANTES ===")
        
        # Compter les valeurs manquantes par colonne
        missing_stats = []
        for col in df.columns:
            missing_count = df[col].isnull().sum()
            total_count = len(df)
            missing_pct = (missing_count / total_count) * 100
            
            # Analyser les types de valeurs non-manquantes
            non_missing = df[col].dropna()
            if len(non_missing) > 0:
                if col in ['Date', 'Mois', 'Heure_Echantillon', 'ID_Station', 'Phase', 'Type_Filtre']:
                    data_type = 'Métadonnée'
                elif non_missing.dtype in ['float64', 'int64']:
                    data_type = 'Numérique'
                else:
                    data_type = 'Texte'
            else:
                data_type = 'Vide'
            
            missing_stats.append({
                'Colonne': col,
                'Valeurs_Manquantes': missing_count,
                'Total': total_count,
                'Pourcentage_Manquant': missing_pct,
                'Type_Donnee': data_type
            })
        
        missing_df = pd.DataFrame(missing_stats)
        missing_df = missing_df.sort_values('Pourcentage_Manquant', ascending=False)
        
        # Afficher le top 10 des colonnes avec le plus de valeurs manquantes
        print("Top 10 des colonnes avec le plus de valeurs manquantes :")
        print(missing_df.head(10)[['Colonne', 'Valeurs_Manquantes', 'Pourcentage_Manquant']].to_string(index=False))
        
        # Statistiques globales
        colonnes_numeriques = missing_df[missing_df['Type_Donnee'] == 'Numérique']
        if len(colonnes_numeriques) > 0:
            print(f"\nColonnes numériques : {len(colonnes_numeriques)}")
            print(f"Valeurs manquantes moyennes (paramètres numériques) : {colonnes_numeriques['Pourcentage_Manquant'].mean():.1f}%")
        
        return missing_df
    
    def handle_missing_values(self, df, strategy='hybrid'):
        """
        Gère les valeurs manquantes selon différentes stratégies
        
        Stratégies :
        - 'conservative' : Garde les NaN, supprime les lignes complètement vides
        - 'aggressive' : Supprime les colonnes avec >80% de valeurs manquantes
        - 'interpolate' : Interpole les valeurs manquantes pour les données temporelles
        - 'hybrid' : Approche intelligente combinée (RECOMMANDÉE)
        """
        print(f"\n=== GESTION DES VALEURS MANQUANTES (stratégie: {strategy}) ===")
        
        df_clean = df.copy()
        initial_shape = df_clean.shape
        
        if strategy == 'conservative':
            # Supprimer les lignes complètement vides (toutes les valeurs numériques sont NaN)
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            df_clean = df_clean.dropna(subset=numeric_cols, how='all')
            print(f"Lignes complètement vides supprimées : {initial_shape[0] - len(df_clean)}")
            
        elif strategy == 'aggressive':
            # Supprimer les colonnes avec plus de 80% de valeurs manquantes
            threshold = 0.8
            cols_to_drop = []
            for col in df_clean.columns:
                if col not in ['ID_Station', 'Phase', 'Type_Filtre', 'Date']:  # Garder les métadonnées importantes
                    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
                    if missing_pct > threshold:
                        cols_to_drop.append(col)
            
            df_clean = df_clean.drop(columns=cols_to_drop)
            print(f"Colonnes supprimées (>{threshold*100}% manquantes) : {len(cols_to_drop)}")
            if cols_to_drop:
                print(f"Colonnes supprimées : {cols_to_drop}")
            
        elif strategy == 'interpolate':
            # Interpolation pour les données temporelles
            if 'Date' in df_clean.columns:
                df_clean = df_clean.sort_values('Date')
                numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
                
                for col in numeric_cols:
                    if col.endswith('_mg_L') or col.endswith('_C') or col.endswith('_pct'):
                        # Interpolation linéaire pour les paramètres de qualité
                        df_clean[col] = df_clean.groupby(['Phase', 'ID_Filtre'])[col].transform(
                            lambda x: x.interpolate(method='linear', limit=2)
                        )
        
        elif strategy == 'hybrid':
            # APPROCHE HYBRIDE - La meilleure pour les données de qualité d'eau
            print("Approche hybride intelligente...")
            
            # Étape 1: Supprimer les lignes complètement vides
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            rows_before = len(df_clean)
            df_clean = df_clean.dropna(subset=numeric_cols, how='all')
            rows_removed = rows_before - len(df_clean)
            print(f"  → Lignes vides supprimées : {rows_removed}")
            
            # Étape 2: Identifier les colonnes très problématiques (>95% manquantes)
            very_empty_cols = []
            for col in df_clean.columns:
                if col not in ['ID_Station', 'Phase', 'Type_Filtre', 'Date', 'Mois']:
                    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
                    if missing_pct > 0.95:  # Plus strict que aggressive
                        very_empty_cols.append(col)
            
            if very_empty_cols:
                df_clean = df_clean.drop(columns=very_empty_cols)
                print(f"  → Colonnes quasi-vides supprimées (>95% manquantes) : {len(very_empty_cols)}")
                print(f"    Colonnes : {very_empty_cols}")
            
            # Étape 3: Interpolation douce pour les paramètres physiques stables (pH, Température)
            stable_params = ['Temperature_C', 'pH', 'Conductivite_uS_cm']
            interpolated_count = 0
            
            for param in stable_params:
                if param in df_clean.columns and 'Date' in df_clean.columns:
                    # Interpolation seulement si max 1 valeur manquante consécutive
                    df_clean = df_clean.sort_values(['Date', 'Phase', 'ID_Filtre'])
                    original_nulls = df_clean[param].isnull().sum()
                    
                    df_clean[param] = df_clean.groupby(['Phase', 'ID_Filtre'])[param].transform(
                        lambda x: x.interpolate(method='linear', limit=1)  # Max 1 trou
                    )
                    
                    final_nulls = df_clean[param].isnull().sum()
                    if original_nulls > final_nulls:
                        interpolated_count += (original_nulls - final_nulls)
            
            if interpolated_count > 0:
                print(f"  → Valeurs interpolées (paramètres stables) : {interpolated_count}")
            
            # Étape 4: Marquer les valeurs estimées vs mesurées
            df_clean['Contient_Valeurs_Estimees'] = False
            for col in df_clean.select_dtypes(include=[np.number]).columns:
                if col not in ['ID_Station']:
                    # Marquer les lignes où au moins 30% des paramètres principaux sont manquants
                    main_params = ['DBO5_mg_L', 'DCO_mg_L', 'MES_mg_L', 'pH', 'Temperature_C']
                    available_main = [p for p in main_params if p in df_clean.columns]
                    
                    if available_main:
                        missing_main = df_clean[available_main].isnull().sum(axis=1)
                        df_clean.loc[missing_main >= len(available_main) * 0.3, 'Contient_Valeurs_Estimees'] = True
        
        final_shape = df_clean.shape
        print(f"Shape avant : {initial_shape}")
        print(f"Shape après : {final_shape}")
        print(f"Données conservées : {(final_shape[0]/initial_shape[0]*100):.1f}% des lignes, "
              f"{(final_shape[1]/initial_shape[1]*100):.1f}% des colonnes")
        
        return df_clean
        """
        Post-traitement du dataset unifié
        """
        print("\n=== POST-TRAITEMENT ===")
        
        # Réorganiser les colonnes
        priority_cols = ['ID_Station', 'Phase', 'Type_Filtre', 'ID_Filtre', 'Date', 'Mois', 'Heure_Echantillon']
        other_cols = [col for col in df.columns if col not in priority_cols]
        df = df[priority_cols + other_cols]
        
        # Trier par date
        if 'Date' in df.columns:
            df = df.sort_values(['Date', 'Phase', 'ID_Filtre']).reset_index(drop=True)
        
        # Statistiques de nettoyage
        total_cells = df.shape[0] * df.shape[1]
        missing_cells = df.isnull().sum().sum()
        missing_pct = (missing_cells / total_cells) * 100
        
        print(f"Données manquantes : {missing_cells}/{total_cells} ({missing_pct:.1f}%)")
        
        # Résumé par phase
        phase_summary = df.groupby('Phase').size()
        print("Répartition par phase :")
        for phase, count in phase_summary.items():
            print(f"  - {phase}: {count} lignes")
        
        return df
    
    def create_performance_analysis(self, df_unified, performance_sheets):
        """
        Analyse les données de performance si disponibles
        """
        print("\n=== ANALYSE DES PERFORMANCES ===")
        
        # Créer des paires entrée-sortie pour calcul des rendements
        paires_filtres = {
            'FV1': {
                'entree': df_unified[(df_unified['Phase'] == 'Entree') & 
                                   (df_unified['ID_Filtre'].isin(['FV1', 'General']))],
                'sortie': df_unified[(df_unified['Phase'] == 'Sortie') & 
                                   (df_unified['ID_Filtre'] == 'FV1')]
            },
            'FV2': {
                'entree': df_unified[(df_unified['Phase'] == 'Entree') & 
                                   (df_unified['ID_Filtre'].isin(['FV2', 'General']))],
                'sortie': df_unified[(df_unified['Phase'] == 'Sortie') & 
                                   (df_unified['ID_Filtre'] == 'FV2')]
            }
        }
        
        performance_results = []
        
        for filtre_id, data in paires_filtres.items():
            if len(data['entree']) > 0 and len(data['sortie']) > 0:
                print(f"Calcul des rendements pour {filtre_id}")
                
                # Fusionner entrée et sortie sur la date
                merged = pd.merge(
                    data['entree'], 
                    data['sortie'], 
                    on='Date', 
                    suffixes=('_entree', '_sortie'),
                    how='inner'
                )
                
                if len(merged) > 0:
                    # Calculer les rendements pour les paramètres clés
                    parametres = ['DBO5_mg_L', 'DCO_mg_L', 'MES_mg_L', 'Azote_Total_mg_L', 'Phosphates_mg_L']
                    
                    for param in parametres:
                        col_entree = f"{param}_entree"
                        col_sortie = f"{param}_sortie"
                        
                        if col_entree in merged.columns and col_sortie in merged.columns:
                            merged[f"Rendement_{param.replace('_mg_L', '')}_pct"] = (
                                (merged[col_entree] - merged[col_sortie]) / 
                                merged[col_entree] * 100
                            ).round(2)
                    
                    performance_results.append(merged)
        
        return performance_results

def main():
    """
    Fonction principale
    """
    # Vérifier que le fichier existe
    excel_file = "Excel donnés qualité UGB.xlsx"
    
    if not Path(excel_file).exists():
        print(f"❌ ERREUR : Fichier '{excel_file}' introuvable !")
        print("📁 Fichiers Excel trouvés dans le répertoire actuel :")
        excel_files = list(Path('.').glob('*.xlsx'))
        if excel_files:
            for i, file in enumerate(excel_files, 1):
                print(f"   {i}. {file.name}")
            print("\n💡 Conseil : Vérifiez le nom exact du fichier ci-dessus")
        else:
            print("   Aucun fichier .xlsx trouvé")
        return
    
    # Initialiser le processeur
    processor = UGBDataProcessor()
    
    # Traiter les données
    result = processor.create_unified_dataset(excel_file)
    
    if result is None:
        print("Échec du traitement")
        return
    
    df_unified, performance_sheets, missing_analysis = result
    
    # Afficher un aperçu
    print("\n=== APERÇU DU DATASET FINAL ===")
    print(f"Shape: {df_unified.shape}")
    print("\nPremières lignes :")
    print(df_unified.head())
    
    print("\nColonnes disponibles :")
    for i, col in enumerate(df_unified.columns):
        print(f"{i+1:2d}. {col}")
    
    # Sauvegarder le dataset principal en CSV
    output_file_csv = "UGB_Sanar_Station_Dataset_Clean.csv"
    df_unified.to_csv(output_file_csv, index=False, encoding='utf-8')
    print(f"✓ Dataset principal sauvegardé : {output_file_csv}")
    
    # Sauvegarder l'analyse des valeurs manquantes en CSV
    missing_file_csv = "UGB_Missing_Values_Analysis.csv"
    missing_analysis.to_csv(missing_file_csv, index=False, encoding='utf-8')
    print(f"✓ Analyse des valeurs manquantes : {missing_file_csv}")
    
    # Analyser les performances si demandé
    performance_results = processor.create_performance_analysis(df_unified, performance_sheets)
    
    if performance_results:
        df_performance = pd.concat(performance_results, ignore_index=True)
        performance_file_csv = "UGB_Performance_Analysis.csv"
        df_performance.to_csv(performance_file_csv, index=False, encoding='utf-8')
        print(f"✓ Analyse des performances : {performance_file_csv}")
    
    print("\n=== TRAITEMENT TERMINÉ ===")

if __name__ == "__main__":
    main()

=== DÉBUT DU PRÉTRAITEMENT UGB ===
Nombre de feuilles chargées : 13
Traitement de la feuille : Entree 1
  ✓ Entree 1: 19 lignes
Traitement de la feuille : Entree FV1
  ✓ Entree FV1: 19 lignes
Traitement de la feuille : SFV1a
  ✓ SFV1a: 19 lignes
Traitement de la feuille : SFV1b
  ✓ SFV1b: 19 lignes
Traitement de la feuille : SFV1c
  ✓ SFV1c: 19 lignes
Traitement de la feuille : Entréé FV2
  ✓ Entréé FV2: 19 lignes
Traitement de la feuille : SFV2a
  ✓ SFV2a: 19 lignes
Traitement de la feuille : SFV2b
  ✓ SFV2b: 19 lignes
Traitement de la feuille : SFHa
  ✓ SFHa: 19 lignes
Traitement de la feuille : SFHb
  ✓ SFHb: 19 lignes

Dataset unifié créé : 190 lignes, 34 colonnes

=== ANALYSE DES VALEURS MANQUANTES ===
Top 10 des colonnes avec le plus de valeurs manquantes :
       Colonne  Valeurs_Manquantes  Pourcentage_Manquant
ENTREE station                 190                 100.0
    Unnamed: 2                 190                 100.0
    Entrée FV1                 190                 100.

AttributeError: 'UGBDataProcessor' object has no attribute 'post_process'

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

class UGBDataProcessor:
    """
    Classe pour le prétraitement des données de qualité d'eau UGB
    """
    
    def __init__(self):
        # Mapping pour uniformiser les noms de colonnes
        self.column_mapping = {
            # Colonnes temporelles
            'Mois': 'Mois',
            'Data': 'Date',
            'Heure echantillon ': 'Heure_Echantillon',
            'Heure echantillon /observations': 'Heure_Echantillon',
            
            # Paramètres hydrauliques
            'Debit entree (m3/jour)': 'Debit_Entree_m3_jour',
            'Charge hydraulique (cm/jour)': 'Charge_Hydraulique_cm_jour',
            
            # Paramètres physico-chimiques
            'Temperature (ºC)': 'Temperature_C',
            'pH': 'pH',
            'CE (µSm/cm)': 'Conductivite_uS_cm',
            'CE (µsm/cm)': 'Conductivite_uS_cm',
            'Redox': 'Potentiel_Redox_mV',
            
            # Paramètres de pollution
            'DBO5 (mg/L)': 'DBO5_mg_L',
            'DCO (mg/L)': 'DCO_mg_L',
            'MeS (mg/L)': 'MES_mg_L',
            'MVS (%)': 'MVS_pct',
            
            # Paramètres azotés (TOUTES LES VARIANTES vers le même nom)
            'Nitrates (mgNO3-/l)': 'Nitrates_mg_L',
            'Ammonium (mgNH4-/l) ': 'Ammonium_mg_L',
            'Azot total (mgN/l)': 'Azote_Total_mg_L',      # Variante 1
            'Azote total (mgN/l)': 'Azote_Total_mg_L',     # Variante 2 
            'Azote toal (mgN/l)': 'Azote_Total_mg_L',      # Variante 3 (typo)
            
            # Paramètres phosphorés
            'Phosphates (mgPO4-/l)': 'Phosphates_mg_L',
            
            # Paramètres microbiologiques
            'Coliformes fécaux (CFU/100ml)': 'Coliformes_Fecaux_CFU_100ml',
            'Oeufs helmint': 'Oeufs_Helminthes',
            
            # Autres
            'Huiles et graisses': 'Huiles_Graisses',
            'Opération': 'Operation',
            'Observation': 'Observations'
        }
        
        # Définition des types de feuilles et leurs caractéristiques
        self.sheet_config = {
            'entree': {
                'sheets': ['Entree 1', 'Entree FV1', 'Entréé FV2'],
                'phase': 'Entree',
                'description': 'Eaux d\'entrée'
            },
            'sortie_fv': {
                'sheets': ['SFV1a', 'SFV1b', 'SFV1c', 'SFV2a', 'SFV2b'],
                'phase': 'Sortie',
                'type_filtre': 'Filtre_Vertical',
                'description': 'Sorties filtres verticaux'
            },
            'sortie_fh': {
                'sheets': ['SFHa', 'SFHb'],
                'phase': 'Sortie',
                'type_filtre': 'Filtre_Horizontal',
                'description': 'Sorties filtres horizontaux'
            },
            'performance': {
                'sheets': ['% él FV1', '% él FV2', '% él de la station'],
                'phase': 'Performance',
                'description': 'Données de performance'
            }
        }
    
    def clean_value(self, x):
        """
        Nettoie une valeur individuelle
        Gère spécifiquement : <2000, >500, "pas de donnée", etc.
        """
        if pd.isna(x):
            return np.nan
        
        if isinstance(x, str):
            x = x.strip()
            
            # Gérer les valeurs spéciales (valeurs manquantes)
            missing_indicators = ['pas de donnée', 'nd', 'n.d.', '-', 'absent', '', 'nan', 'null']
            if any(term in x.lower() for term in missing_indicators):
                return np.nan
            
            # Gérer les valeurs avec > (ex: ">500" devient 500)
            # Interprétation: valeur minimale possible
            if '>' in x:
                try:
                    numeric_part = x.replace('>', '').strip()
                    return float(numeric_part)
                except ValueError:
                    return np.nan
            
            # Gérer les valeurs avec < (ex: "<2000" devient 2000) 
            # Interprétation: valeur maximale possible (limite de détection)
            if '<' in x:
                try:
                    numeric_part = x.replace('<', '').strip()
                    return float(numeric_part)
                except ValueError:
                    return np.nan
            
            # Gérer d'autres formats possibles
            # Retirer les espaces et caractères parasites
            x = x.replace(',', '.').replace(' ', '')
            
            # Essayer de convertir en numérique
            try:
                return float(x)
            except ValueError:
                # Si ce n'est pas numérique, retourner tel quel (texte)
                return x if x else np.nan
        
        return x
    
    def standardize_columns(self, df, sheet_name):
        """
        Standardise les noms de colonnes d'un DataFrame
        """
        df_clean = df.copy()
        
        # Renommer les colonnes selon le mapping
        df_clean.rename(columns=self.column_mapping, inplace=True)
        
        # Ajouter les métadonnées
        df_clean['ID_Station'] = 'Sanar_Station'
        df_clean['Nom_Feuille'] = sheet_name
        
        # Déterminer le type et la phase selon la feuille
        for config_type, config in self.sheet_config.items():
            if sheet_name in config['sheets']:
                df_clean['Phase'] = config['phase']
                if 'type_filtre' in config:
                    df_clean['Type_Filtre'] = config['type_filtre']
                else:
                    df_clean['Type_Filtre'] = 'Non_Applicable'
                break
        
        # Identifier le filtre spécifique
        if 'FV1' in sheet_name:
            df_clean['ID_Filtre'] = 'FV1'
        elif 'FV2' in sheet_name:
            df_clean['ID_Filtre'] = 'FV2'
        elif 'FH' in sheet_name:
            df_clean['ID_Filtre'] = 'FH'
        else:
            df_clean['ID_Filtre'] = 'General'
        
        return df_clean
    
    def process_sheet(self, df, sheet_name):
        """
        Traite une feuille individuelle
        """
        print(f"Traitement de la feuille : {sheet_name}")
        
        # Standardiser les colonnes
        df_processed = self.standardize_columns(df, sheet_name)
        
        # Nettoyer les valeurs (sauf les métadonnées)
        metadata_cols = ['ID_Station', 'Nom_Feuille', 'Phase', 'Type_Filtre', 'ID_Filtre']
        
        for col in df_processed.columns:
            if col not in metadata_cols:
                df_processed[col] = df_processed[col].apply(self.clean_value)
        
        # Traiter la date
        if 'Date' in df_processed.columns:
            df_processed['Date'] = pd.to_datetime(df_processed['Date'], errors='coerce')
        
        return df_processed
    
    def create_unified_dataset(self, excel_file_path):
        """
        Crée un dataset unifié à partir du fichier Excel
        """
        print("=== DÉBUT DU PRÉTRAITEMENT UGB ===")
        
        # 1. Charger toutes les feuilles
        try:
            df_dict = pd.read_excel(excel_file_path, sheet_name=None)
            print(f"Nombre de feuilles chargées : {len(df_dict)}")
        except Exception as e:
            print(f"Erreur lors du chargement : {e}")
            return None
        
        # 2. Traiter chaque feuille (exclure les feuilles de performance pour l'instant)
        processed_sheets = []
        performance_sheets = []
        
        for sheet_name, data in df_dict.items():
            if sheet_name.startswith('% él'):
                # Traiter séparément les feuilles de performance
                performance_sheets.append((sheet_name, data))
                continue
            
            try:
                processed_df = self.process_sheet(data, sheet_name)
                processed_sheets.append(processed_df)
                print(f"  ✓ {sheet_name}: {len(processed_df)} lignes")
            except Exception as e:
                print(f"  ✗ Erreur avec {sheet_name}: {e}")
        
        # 3. Combiner toutes les feuilles
        if processed_sheets:
            df_unified = pd.concat(processed_sheets, ignore_index=True, sort=False)
            print(f"\nDataset unifié créé : {len(df_unified)} lignes, {len(df_unified.columns)} colonnes")
        else:
            print("Aucune feuille traitée avec succès")
            return None
        
        # 4. Analyser les valeurs manquantes
        missing_analysis = self.analyze_missing_values(df_unified)
        
        # 5. Gérer les valeurs manquantes 
        # CHOISIR UNE SEULE STRATÉGIE selon vos besoins :
        
        # Option 1 - HYBRID (Recommandée - approche intelligente)
        df_unified = self.handle_missing_values(df_unified, strategy='hybrid')
        
        # Option 2 - CONSERVATIVE (décommenter si vous voulez garder toutes les données)
        # df_unified = self.handle_missing_values(df_unified, strategy='conservative')
        
        # Option 3 - AGGRESSIVE (décommenter si vous voulez supprimer les colonnes très incomplètes)
        # df_unified = self.handle_missing_values(df_unified, strategy='aggressive')
        
        # Option 4 - INTERPOLATE (décommenter si vous voulez interpoler les séries temporelles)  
        # df_unified = self.handle_missing_values(df_unified, strategy='interpolate')
        
        # 6. Post-traitement final
        df_unified = self.post_process(df_unified)
        
        return df_unified, performance_sheets, missing_analysis
    
    def analyze_missing_values(self, df):
        """
        Analyse détaillée des valeurs manquantes
        """
        print("\n=== ANALYSE DES VALEURS MANQUANTES ===")
        
        # Compter les valeurs manquantes par colonne
        missing_stats = []
        for col in df.columns:
            missing_count = df[col].isnull().sum()
            total_count = len(df)
            missing_pct = (missing_count / total_count) * 100
            
            # Analyser les types de valeurs non-manquantes
            non_missing = df[col].dropna()
            if len(non_missing) > 0:
                if col in ['Date', 'Mois', 'Heure_Echantillon', 'ID_Station', 'Phase', 'Type_Filtre']:
                    data_type = 'Métadonnée'
                elif non_missing.dtype in ['float64', 'int64']:
                    data_type = 'Numérique'
                else:
                    data_type = 'Texte'
            else:
                data_type = 'Vide'
            
            missing_stats.append({
                'Colonne': col,
                'Valeurs_Manquantes': missing_count,
                'Total': total_count,
                'Pourcentage_Manquant': missing_pct,
                'Type_Donnee': data_type
            })
        
        missing_df = pd.DataFrame(missing_stats)
        missing_df = missing_df.sort_values('Pourcentage_Manquant', ascending=False)
        
        # Afficher le top 10 des colonnes avec le plus de valeurs manquantes
        print("Top 10 des colonnes avec le plus de valeurs manquantes :")
        print(missing_df.head(10)[['Colonne', 'Valeurs_Manquantes', 'Pourcentage_Manquant']].to_string(index=False))
        
        # Statistiques globales
        colonnes_numeriques = missing_df[missing_df['Type_Donnee'] == 'Numérique']
        if len(colonnes_numeriques) > 0:
            print(f"\nColonnes numériques : {len(colonnes_numeriques)}")
            print(f"Valeurs manquantes moyennes (paramètres numériques) : {colonnes_numeriques['Pourcentage_Manquant'].mean():.1f}%")
        
        return missing_df
    
    def handle_missing_values(self, df, strategy='hybrid'):
        """
        Gère les valeurs manquantes selon différentes stratégies
        
        Stratégies :
        - 'conservative' : Garde les NaN, supprime les lignes complètement vides
        - 'aggressive' : Supprime les colonnes avec >80% de valeurs manquantes
        - 'interpolate' : Interpole les valeurs manquantes pour les données temporelles
        - 'hybrid' : Approche intelligente combinée (RECOMMANDÉE)
        """
        print(f"\n=== GESTION DES VALEURS MANQUANTES (stratégie: {strategy}) ===")
        
        df_clean = df.copy()
        initial_shape = df_clean.shape
        
        if strategy == 'conservative':
            # Supprimer les lignes complètement vides (toutes les valeurs numériques sont NaN)
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            df_clean = df_clean.dropna(subset=numeric_cols, how='all')
            print(f"Lignes complètement vides supprimées : {initial_shape[0] - len(df_clean)}")
            
        elif strategy == 'aggressive':
            # Supprimer les colonnes avec plus de 80% de valeurs manquantes
            threshold = 0.8
            cols_to_drop = []
            for col in df_clean.columns:
                if col not in ['ID_Station', 'Phase', 'Type_Filtre', 'Date']:  # Garder les métadonnées importantes
                    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
                    if missing_pct > threshold:
                        cols_to_drop.append(col)
            
            df_clean = df_clean.drop(columns=cols_to_drop)
            print(f"Colonnes supprimées (>{threshold*100}% manquantes) : {len(cols_to_drop)}")
            if cols_to_drop:
                print(f"Colonnes supprimées : {cols_to_drop}")
            
        elif strategy == 'interpolate':
            # Interpolation pour les données temporelles
            if 'Date' in df_clean.columns:
                df_clean = df_clean.sort_values('Date')
                numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
                
                for col in numeric_cols:
                    if col.endswith('_mg_L') or col.endswith('_C') or col.endswith('_pct'):
                        # Interpolation linéaire pour les paramètres de qualité
                        df_clean[col] = df_clean.groupby(['Phase', 'ID_Filtre'])[col].transform(
                            lambda x: x.interpolate(method='linear', limit=2)
                        )
        
        elif strategy == 'hybrid':
            # APPROCHE HYBRIDE - La meilleure pour les données de qualité d'eau
            print("Approche hybride intelligente...")
            
            # Étape 1: Supprimer les lignes complètement vides
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            rows_before = len(df_clean)
            df_clean = df_clean.dropna(subset=numeric_cols, how='all')
            rows_removed = rows_before - len(df_clean)
            print(f"  → Lignes vides supprimées : {rows_removed}")
            
            # Étape 2: Identifier les colonnes très problématiques (>95% manquantes)
            very_empty_cols = []
            for col in df_clean.columns:
                if col not in ['ID_Station', 'Phase', 'Type_Filtre', 'Date', 'Mois']:
                    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
                    if missing_pct > 0.95:  # Plus strict que aggressive
                        very_empty_cols.append(col)
            
            if very_empty_cols:
                df_clean = df_clean.drop(columns=very_empty_cols)
                print(f"  → Colonnes quasi-vides supprimées (>95% manquantes) : {len(very_empty_cols)}")
                print(f"    Colonnes : {very_empty_cols}")
            
            # Étape 3: Interpolation douce pour les paramètres physiques stables (pH, Température)
            stable_params = ['Temperature_C', 'pH', 'Conductivite_uS_cm']
            interpolated_count = 0
            
            for param in stable_params:
                if param in df_clean.columns and 'Date' in df_clean.columns:
                    # Interpolation seulement si max 1 valeur manquante consécutive
                    df_clean = df_clean.sort_values(['Date', 'Phase', 'ID_Filtre'])
                    original_nulls = df_clean[param].isnull().sum()
                    
                    df_clean[param] = df_clean.groupby(['Phase', 'ID_Filtre'])[param].transform(
                        lambda x: x.interpolate(method='linear', limit=1)  # Max 1 trou
                    )
                    
                    final_nulls = df_clean[param].isnull().sum()
                    if original_nulls > final_nulls:
                        interpolated_count += (original_nulls - final_nulls)
            
            if interpolated_count > 0:
                print(f"  → Valeurs interpolées (paramètres stables) : {interpolated_count}")
            
            # Étape 4: Marquer les valeurs estimées vs mesurées
            df_clean['Contient_Valeurs_Estimees'] = False
            for col in df_clean.select_dtypes(include=[np.number]).columns:
                if col not in ['ID_Station']:
                    # Marquer les lignes où au moins 30% des paramètres principaux sont manquants
                    main_params = ['DBO5_mg_L', 'DCO_mg_L', 'MES_mg_L', 'pH', 'Temperature_C']
                    available_main = [p for p in main_params if p in df_clean.columns]
                    
                    if available_main:
                        missing_main = df_clean[available_main].isnull().sum(axis=1)
                        df_clean.loc[missing_main >= len(available_main) * 0.3, 'Contient_Valeurs_Estimees'] = True
        
        final_shape = df_clean.shape
        print(f"Shape avant : {initial_shape}")
        print(f"Shape après : {final_shape}")
        print(f"Données conservées : {(final_shape[0]/initial_shape[0]*100):.1f}% des lignes, "
              f"{(final_shape[1]/initial_shape[1]*100):.1f}% des colonnes")
        
        return df_clean
        """
        Post-traitement du dataset unifié
        """
        print("\n=== POST-TRAITEMENT ===")
        
        # Réorganiser les colonnes
        priority_cols = ['ID_Station', 'Phase', 'Type_Filtre', 'ID_Filtre', 'Date', 'Mois', 'Heure_Echantillon']
        other_cols = [col for col in df.columns if col not in priority_cols]
        df = df[priority_cols + other_cols]
        
        # Trier par date
        if 'Date' in df.columns:
            df = df.sort_values(['Date', 'Phase', 'ID_Filtre']).reset_index(drop=True)
        
        # Statistiques de nettoyage
        total_cells = df.shape[0] * df.shape[1]
        missing_cells = df.isnull().sum().sum()
        missing_pct = (missing_cells / total_cells) * 100
        
        print(f"Données manquantes : {missing_cells}/{total_cells} ({missing_pct:.1f}%)")
        
        # Résumé par phase
        phase_summary = df.groupby('Phase').size()
        print("Répartition par phase :")
        for phase, count in phase_summary.items():
            print(f"  - {phase}: {count} lignes")
        
        return df
    
    def create_performance_analysis(self, df_unified, performance_sheets):
        """
        Analyse les données de performance si disponibles
        """
        print("\n=== ANALYSE DES PERFORMANCES ===")
        
        # Créer des paires entrée-sortie pour calcul des rendements
        paires_filtres = {
            'FV1': {
                'entree': df_unified[(df_unified['Phase'] == 'Entree') & 
                                   (df_unified['ID_Filtre'].isin(['FV1', 'General']))],
                'sortie': df_unified[(df_unified['Phase'] == 'Sortie') & 
                                   (df_unified['ID_Filtre'] == 'FV1')]
            },
            'FV2': {
                'entree': df_unified[(df_unified['Phase'] == 'Entree') & 
                                   (df_unified['ID_Filtre'].isin(['FV2', 'General']))],
                'sortie': df_unified[(df_unified['Phase'] == 'Sortie') & 
                                   (df_unified['ID_Filtre'] == 'FV2')]
            }
        }
        
        performance_results = []
        
        for filtre_id, data in paires_filtres.items():
            if len(data['entree']) > 0 and len(data['sortie']) > 0:
                print(f"Calcul des rendements pour {filtre_id}")
                
                # Fusionner entrée et sortie sur la date
                merged = pd.merge(
                    data['entree'], 
                    data['sortie'], 
                    on='Date', 
                    suffixes=('_entree', '_sortie'),
                    how='inner'
                )
                
                if len(merged) > 0:
                    # Calculer les rendements pour les paramètres clés
                    parametres = ['DBO5_mg_L', 'DCO_mg_L', 'MES_mg_L', 'Azote_Total_mg_L', 'Phosphates_mg_L']
                    
                    for param in parametres:
                        col_entree = f"{param}_entree"
                        col_sortie = f"{param}_sortie"
                        
                        if col_entree in merged.columns and col_sortie in merged.columns:
                            merged[f"Rendement_{param.replace('_mg_L', '')}_pct"] = (
                                (merged[col_entree] - merged[col_sortie]) / 
                                merged[col_entree] * 100
                            ).round(2)
                    
                    performance_results.append(merged)
        
        return performance_results

def main():
    """
    Fonction principale
    """
    # Vérifier que le fichier existe
    excel_file = "Excel donnés qualité UGB.xlsx"
    
    if not Path(excel_file).exists():
        print(f"❌ ERREUR : Fichier '{excel_file}' introuvable !")
        print("📁 Fichiers Excel trouvés dans le répertoire actuel :")
        excel_files = list(Path('.').glob('*.xlsx'))
        if excel_files:
            for i, file in enumerate(excel_files, 1):
                print(f"   {i}. {file.name}")
            print("\n💡 Conseil : Vérifiez le nom exact du fichier ci-dessus")
        else:
            print("   Aucun fichier .xlsx trouvé")
        return
    
    # Initialiser le processeur
    processor = UGBDataProcessor()
    
    # Traiter les données
    result = processor.create_unified_dataset(excel_file)
    
    if result is None:
        print("Échec du traitement")
        return
    
    df_unified, performance_sheets, missing_analysis = result
    
    # Afficher un aperçu
    print("\n=== APERÇU DU DATASET FINAL ===")
    print(f"Shape: {df_unified.shape}")
    print("\nPremières lignes :")
    print(df_unified.head())
    
    print("\nColonnes disponibles :")
    for i, col in enumerate(df_unified.columns):
        print(f"{i+1:2d}. {col}")
    
    # Sauvegarder le dataset principal en CSV
    output_file_csv = "UGB_Sanar_Station_Dataset_Clean.csv"
    df_unified.to_csv(output_file_csv, index=False, encoding='utf-8')
    print(f"✓ Dataset principal sauvegardé : {output_file_csv}")
    
    # Sauvegarder l'analyse des valeurs manquantes en CSV
    missing_file_csv = "UGB_Missing_Values_Analysis.csv"
    missing_analysis.to_csv(missing_file_csv, index=False, encoding='utf-8')
    print(f"✓ Analyse des valeurs manquantes : {missing_file_csv}")
    
    # Analyser les performances si demandé
    performance_results = processor.create_performance_analysis(df_unified, performance_sheets)
    
    if performance_results:
        df_performance = pd.concat(performance_results, ignore_index=True)
        performance_file_csv = "UGB_Performance_Analysis.csv"
        df_performance.to_csv(performance_file_csv, index=False, encoding='utf-8')
        print(f"✓ Analyse des performances : {performance_file_csv}")
    
    print("\n=== TRAITEMENT TERMINÉ ===")

if __name__ == "__main__":
    main()

=== DÉBUT DU PRÉTRAITEMENT UGB ===
Nombre de feuilles chargées : 13
Traitement de la feuille : Entree 1
  ✓ Entree 1: 19 lignes
Traitement de la feuille : Entree FV1
  ✓ Entree FV1: 19 lignes
Traitement de la feuille : SFV1a
  ✓ SFV1a: 19 lignes
Traitement de la feuille : SFV1b
  ✓ SFV1b: 19 lignes
Traitement de la feuille : SFV1c
  ✓ SFV1c: 19 lignes
Traitement de la feuille : Entréé FV2
  ✓ Entréé FV2: 19 lignes
Traitement de la feuille : SFV2a
  ✓ SFV2a: 19 lignes
Traitement de la feuille : SFV2b
  ✓ SFV2b: 19 lignes
Traitement de la feuille : SFHa
  ✓ SFHa: 19 lignes
Traitement de la feuille : SFHb
  ✓ SFHb: 19 lignes

Dataset unifié créé : 190 lignes, 34 colonnes

=== ANALYSE DES VALEURS MANQUANTES ===
Top 10 des colonnes avec le plus de valeurs manquantes :
       Colonne  Valeurs_Manquantes  Pourcentage_Manquant
ENTREE station                 190                 100.0
    Unnamed: 2                 190                 100.0
    Entrée FV1                 190                 100.

AttributeError: 'UGBDataProcessor' object has no attribute 'post_process'

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

class UGBDataProcessor:
    """
    Classe pour le prétraitement des données de qualité d'eau UGB
    """
    
    def __init__(self):
        # Mapping pour uniformiser les noms de colonnes
        self.column_mapping = {
            # Colonnes temporelles
            'Mois': 'Mois',
            'Data': 'Date',
            'Heure echantillon ': 'Heure_Echantillon',
            'Heure echantillon /observations': 'Heure_Echantillon',
            
            # Paramètres hydrauliques
            'Debit entree (m3/jour)': 'Debit_Entree_m3_jour',
            'Charge hydraulique (cm/jour)': 'Charge_Hydraulique_cm_jour',
            
            # Paramètres physico-chimiques
            'Temperature (ºC)': 'Temperature_C',
            'pH': 'pH',
            'CE (µSm/cm)': 'Conductivite_uS_cm',
            'CE (µsm/cm)': 'Conductivite_uS_cm',
            'Redox': 'Potentiel_Redox_mV',
            
            # Paramètres de pollution
            'DBO5 (mg/L)': 'DBO5_mg_L',
            'DCO (mg/L)': 'DCO_mg_L',
            'MeS (mg/L)': 'MES_mg_L',
            'MVS (%)': 'MVS_pct',
            
            # Paramètres azotés (TOUTES LES VARIANTES vers le même nom)
            'Nitrates (mgNO3-/l)': 'Nitrates_mg_L',
            'Ammonium (mgNH4-/l) ': 'Ammonium_mg_L',
            'Azot total (mgN/l)': 'Azote_Total_mg_L',      # Variante 1
            'Azote total (mgN/l)': 'Azote_Total_mg_L',     # Variante 2 
            'Azote toal (mgN/l)': 'Azote_Total_mg_L',      # Variante 3 (typo)
            
            # Paramètres phosphorés
            'Phosphates (mgPO4-/l)': 'Phosphates_mg_L',
            
            # Paramètres microbiologiques
            'Coliformes fécaux (CFU/100ml)': 'Coliformes_Fecaux_CFU_100ml',
            'Oeufs helmint': 'Oeufs_Helminthes',
            
            # Autres
            'Huiles et graisses': 'Huiles_Graisses',
            'Opération': 'Operation',
            'Observation': 'Observations'
        }
        
        # Définition des types de feuilles et leurs caractéristiques
        self.sheet_config = {
            'entree': {
                'sheets': ['Entree 1', 'Entree FV1', 'Entréé FV2'],
                'phase': 'Entree',
                'description': 'Eaux d\'entrée'
            },
            'sortie_fv': {
                'sheets': ['SFV1a', 'SFV1b', 'SFV1c', 'SFV2a', 'SFV2b'],
                'phase': 'Sortie',
                'type_filtre': 'Filtre_Vertical',
                'description': 'Sorties filtres verticaux'
            },
            'sortie_fh': {
                'sheets': ['SFHa', 'SFHb'],
                'phase': 'Sortie',
                'type_filtre': 'Filtre_Horizontal',
                'description': 'Sorties filtres horizontaux'
            },
            'performance': {
                'sheets': ['% él FV1', '% él FV2', '% él de la station'],
                'phase': 'Performance',
                'description': 'Données de performance'
            }
        }
    
    def clean_value(self, x):
        """
        Nettoie une valeur individuelle
        Gère spécifiquement : <2000, >500, "pas de donnée", etc.
        """
        if pd.isna(x):
            return np.nan
        
        if isinstance(x, str):
            x = x.strip()
            
            # Gérer les valeurs spéciales (valeurs manquantes)
            missing_indicators = ['pas de donnée', 'nd', 'n.d.', '-', 'absent', '', 'nan', 'null']
            if any(term in x.lower() for term in missing_indicators):
                return np.nan
            
            # Gérer les valeurs avec > (ex: ">500" devient 500)
            # Interprétation: valeur minimale possible
            if '>' in x:
                try:
                    numeric_part = x.replace('>', '').strip()
                    return float(numeric_part)
                except ValueError:
                    return np.nan
            
            # Gérer les valeurs avec < (ex: "<2000" devient 2000) 
            # Interprétation: valeur maximale possible (limite de détection)
            if '<' in x:
                try:
                    numeric_part = x.replace('<', '').strip()
                    return float(numeric_part)
                except ValueError:
                    return np.nan
            
            # Gérer d'autres formats possibles
            # Retirer les espaces et caractères parasites
            x = x.replace(',', '.').replace(' ', '')
            
            # Essayer de convertir en numérique
            try:
                return float(x)
            except ValueError:
                # Si ce n'est pas numérique, retourner tel quel (texte)
                return x if x else np.nan
        
        return x
    
    def standardize_columns(self, df, sheet_name):
        """
        Standardise les noms de colonnes d'un DataFrame
        """
        df_clean = df.copy()
        
        # Renommer les colonnes selon le mapping
        df_clean.rename(columns=self.column_mapping, inplace=True)
        
        # Ajouter les métadonnées
        df_clean['ID_Station'] = 'Sanar_Station'
        df_clean['Nom_Feuille'] = sheet_name
        
        # Déterminer le type et la phase selon la feuille
        for config_type, config in self.sheet_config.items():
            if sheet_name in config['sheets']:
                df_clean['Phase'] = config['phase']
                if 'type_filtre' in config:
                    df_clean['Type_Filtre'] = config['type_filtre']
                else:
                    df_clean['Type_Filtre'] = 'Non_Applicable'
                break
        
        # Identifier le filtre spécifique
        if 'FV1' in sheet_name:
            df_clean['ID_Filtre'] = 'FV1'
        elif 'FV2' in sheet_name:
            df_clean['ID_Filtre'] = 'FV2'
        elif 'FH' in sheet_name:
            df_clean['ID_Filtre'] = 'FH'
        else:
            df_clean['ID_Filtre'] = 'General'
        
        return df_clean
    
    def process_sheet(self, df, sheet_name):
        """
        Traite une feuille individuelle
        """
        print(f"Traitement de la feuille : {sheet_name}")
        
        # Standardiser les colonnes
        df_processed = self.standardize_columns(df, sheet_name)
        
        # Nettoyer les valeurs (sauf les métadonnées)
        metadata_cols = ['ID_Station', 'Nom_Feuille', 'Phase', 'Type_Filtre', 'ID_Filtre']
        
        for col in df_processed.columns:
            if col not in metadata_cols:
                df_processed[col] = df_processed[col].apply(self.clean_value)
        
        # Traiter la date
        if 'Date' in df_processed.columns:
            df_processed['Date'] = pd.to_datetime(df_processed['Date'], errors='coerce')
        
        return df_processed
    
    def analyze_missing_values(self, df):
        """
        Analyse détaillée des valeurs manquantes
        """
        print("\n=== ANALYSE DES VALEURS MANQUANTES ===")
        
        # Compter les valeurs manquantes par colonne
        missing_stats = []
        for col in df.columns:
            missing_count = df[col].isnull().sum()
            total_count = len(df)
            missing_pct = (missing_count / total_count) * 100
            
            # Analyser les types de valeurs non-manquantes
            non_missing = df[col].dropna()
            if len(non_missing) > 0:
                if col in ['Date', 'Mois', 'Heure_Echantillon', 'ID_Station', 'Phase', 'Type_Filtre']:
                    data_type = 'Métadonnée'
                elif non_missing.dtype in ['float64', 'int64']:
                    data_type = 'Numérique'
                else:
                    data_type = 'Texte'
            else:
                data_type = 'Vide'
            
            missing_stats.append({
                'Colonne': col,
                'Valeurs_Manquantes': missing_count,
                'Total': total_count,
                'Pourcentage_Manquant': missing_pct,
                'Type_Donnee': data_type
            })
        
        missing_df = pd.DataFrame(missing_stats)
        missing_df = missing_df.sort_values('Pourcentage_Manquant', ascending=False)
        
        # Afficher le top 10 des colonnes avec le plus de valeurs manquantes
        print(" 10 des colonnes avec le plus de valeurs manquantes :")
        print(missing_df.head(10)[['Colonne', 'Valeurs_Manquantes', 'Pourcentage_Manquant']].to_string(index=False))
        
        # Statistiques globales
        colonnes_numeriques = missing_df[missing_df['Type_Donnee'] == 'Numérique']
        if len(colonnes_numeriques) > 0:
            print(f"\nColonnes numériques : {len(colonnes_numeriques)}")
            print(f"Valeurs manquantes moyennes (paramètres numériques) : {colonnes_numeriques['Pourcentage_Manquant'].mean():.1f}%")
        
        return missing_df
    
    def handle_missing_values(self, df, strategy='hybrid'):
        """
        Gère les valeurs manquantes selon différentes stratégies
        """
        print(f"\n=== GESTION DES VALEURS MANQUANTES (stratégie: {strategy}) ===")
        
        df_clean = df.copy()
        initial_shape = df_clean.shape
        
        if strategy == 'conservative':
            # Supprimer les lignes complètement vides (toutes les valeurs numériques sont NaN)
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            df_clean = df_clean.dropna(subset=numeric_cols, how='all')
            print(f"Lignes complètement vides supprimées : {initial_shape[0] - len(df_clean)}")
            
        elif strategy == 'aggressive':
            # Supprimer les colonnes avec plus de 80% de valeurs manquantes
            threshold = 0.8
            cols_to_drop = []
            for col in df_clean.columns:
                if col not in ['ID_Station', 'Phase', 'Type_Filtre', 'Date']:  # Garder les métadonnées importantes
                    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
                    if missing_pct > threshold:
                        cols_to_drop.append(col)
            
            df_clean = df_clean.drop(columns=cols_to_drop)
            print(f"Colonnes supprimées (>{threshold*100}% manquantes) : {len(cols_to_drop)}")
            if cols_to_drop:
                print(f"Colonnes supprimées : {cols_to_drop}")
                
        elif strategy == 'hybrid':
            # APPROCHE HYBRIDE - La meilleure pour les données de qualité d'eau
            print("Approche hybride intelligente...")
            
            # Étape 1: Supprimer les lignes complètement vides
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            rows_before = len(df_clean)
            df_clean = df_clean.dropna(subset=numeric_cols, how='all')
            rows_removed = rows_before - len(df_clean)
            print(f"  → Lignes vides supprimées : {rows_removed}")
            
            # Étape 2: Identifier les colonnes très problématiques (>95% manquantes)
            very_empty_cols = []
            for col in df_clean.columns:
                if col not in ['ID_Station', 'Phase', 'Type_Filtre', 'Date', 'Mois']:
                    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
                    if missing_pct > 0.95:  # Plus strict que aggressive
                        very_empty_cols.append(col)
            
            if very_empty_cols:
                df_clean = df_clean.drop(columns=very_empty_cols)
                print(f"  → Colonnes quasi-vides supprimées (>95% manquantes) : {len(very_empty_cols)}")
                print(f"    Colonnes : {very_empty_cols}")
            
            # Étape 3: Interpolation douce pour les paramètres physiques stables
            stable_params = ['Temperature_C', 'pH', 'Conductivite_uS_cm']
            interpolated_count = 0
            
            for param in stable_params:
                if param in df_clean.columns and 'Date' in df_clean.columns:
                    # Interpolation seulement si max 1 valeur manquante consécutive
                    df_clean = df_clean.sort_values(['Date', 'Phase', 'ID_Filtre'])
                    original_nulls = df_clean[param].isnull().sum()
                    
                    df_clean[param] = df_clean.groupby(['Phase', 'ID_Filtre'])[param].transform(
                        lambda x: x.interpolate(method='linear', limit=1)  # Max 1 trou
                    )
                    
                    final_nulls = df_clean[param].isnull().sum()
                    if original_nulls > final_nulls:
                        interpolated_count += (original_nulls - final_nulls)
            
            if interpolated_count > 0:
                print(f"  → Valeurs interpolées (paramètres stables) : {interpolated_count}")
            
            # Étape 4: Marquer les valeurs estimées vs mesurées
            df_clean['Contient_Valeurs_Estimees'] = False
            main_params = ['DBO5_mg_L', 'DCO_mg_L', 'MES_mg_L', 'pH', 'Temperature_C']
            available_main = [p for p in main_params if p in df_clean.columns]
            
            if available_main:
                missing_main = df_clean[available_main].isnull().sum(axis=1)
                df_clean.loc[missing_main >= len(available_main) * 0.3, 'Contient_Valeurs_Estimees'] = True
        
        final_shape = df_clean.shape
        print(f"Shape avant : {initial_shape}")
        print(f"Shape après : {final_shape}")
        print(f"Données conservées : {(final_shape[0]/initial_shape[0]*100):.1f}% des lignes, "
              f"{(final_shape[1]/initial_shape[1]*100):.1f}% des colonnes")
        
        return df_clean

    def post_process(self, df):
        """
        Post-traitement du dataset unifié
        """
        print("\n=== POST-TRAITEMENT ===")
        
        # Réorganiser les colonnes par ordre de priorité
        priority_cols = ['ID_Station', 'Phase', 'Type_Filtre', 'ID_Filtre', 'Date', 'Mois', 'Heure_Echantillon']
        available_priority = [col for col in priority_cols if col in df.columns]
        other_cols = [col for col in df.columns if col not in priority_cols]
        df = df[available_priority + other_cols]
        
        # Trier par date si possible
        if 'Date' in df.columns:
            df = df.sort_values(['Date', 'Phase', 'ID_Filtre']).reset_index(drop=True)
        
        # Statistiques de nettoyage finales
        total_cells = df.shape[0] * df.shape[1]
        missing_cells = df.isnull().sum().sum()
        missing_pct = (missing_cells / total_cells) * 100
        
        print(f"Données manquantes finales : {missing_cells}/{total_cells} ({missing_pct:.1f}%)")
        
        # Résumé par phase
        if 'Phase' in df.columns:
            phase_summary = df.groupby('Phase').size()
            print("Répartition par phase :")
            for phase, count in phase_summary.items():
                print(f"  - {phase}: {count} lignes")
        
        return df
    
    def create_unified_dataset(self, excel_file_path):
        """
        Crée un dataset unifié à partir du fichier Excel
        """
        print("=== DÉBUT DU PRÉTRAITEMENT UGB ===")
        
        # 1. Charger toutes les feuilles
        try:
            df_dict = pd.read_excel(excel_file_path, sheet_name=None, header=3)

            print(f"Nombre de feuilles chargées : {len(df_dict)}")
        except Exception as e:
            print(f"Erreur lors du chargement : {e}")
            return None
        
        # 2. Traiter chaque feuille (exclure les feuilles de performance pour l'instant)
        processed_sheets = []
        performance_sheets = []
        
        for sheet_name, data in df_dict.items():
            if sheet_name.startswith('% él'):
                # Traiter séparément les feuilles de performance
                performance_sheets.append((sheet_name, data))
                continue
            
            try:
                processed_df = self.process_sheet(data, sheet_name)
                processed_sheets.append(processed_df)
                print(f"  ✓ {sheet_name}: {len(processed_df)} lignes")
            except Exception as e:
                print(f"  ✗ Erreur avec {sheet_name}: {e}")
        
        # 3. Combiner toutes les feuilles
        if processed_sheets:
            df_unified = pd.concat(processed_sheets, ignore_index=True, sort=False)
            print(f"\nDataset unifié créé : {len(df_unified)} lignes, {len(df_unified.columns)} colonnes")
        else:
            print("Aucune feuille traitée avec succès")
            return None
        
        # 4. Analyser les valeurs manquantes
        missing_analysis = self.analyze_missing_values(df_unified)
        
        # 5. Gérer les valeurs manquantes avec stratégie hybrid
        df_unified = self.handle_missing_values(df_unified, strategy='hybrid')
        
        # 6. Post-traitement final
        df_unified = self.post_process(df_unified)
        
        return df_unified, performance_sheets, missing_analysis

    def create_performance_analysis(self, df_unified, performance_sheets):
        """
        Analyse les données de performance si disponibles
        """
        print("\n=== ANALYSE DES PERFORMANCES ===")
        
        # Créer des paires entrée-sortie pour calcul des rendements
        paires_filtres = {
            'FV1': {
                'entree': df_unified[(df_unified['Phase'] == 'Entree') & 
                                   (df_unified['ID_Filtre'].isin(['FV1', 'General']))],
                'sortie': df_unified[(df_unified['Phase'] == 'Sortie') & 
                                   (df_unified['ID_Filtre'] == 'FV1')]
            },
            'FV2': {
                'entree': df_unified[(df_unified['Phase'] == 'Entree') & 
                                   (df_unified['ID_Filtre'].isin(['FV2', 'General']))],
                'sortie': df_unified[(df_unified['Phase'] == 'Sortie') & 
                                   (df_unified['ID_Filtre'] == 'FV2')]
            }
        }
        
        performance_results = []
        
        for filtre_id, data in paires_filtres.items():
            if len(data['entree']) > 0 and len(data['sortie']) > 0:
                print(f"Calcul des rendements pour {filtre_id}")
                
                # Fusionner entrée et sortie sur la date
                merged = pd.merge(
                    data['entree'], 
                    data['sortie'], 
                    on='Date', 
                    suffixes=('_entree', '_sortie'),
                    how='inner'
                )
                
                if len(merged) > 0:
                    # Calculer les rendements pour les paramètres clés
                    parametres = ['DBO5_mg_L', 'DCO_mg_L', 'MES_mg_L', 'Azote_Total_mg_L', 'Phosphates_mg_L']
                    
                    for param in parametres:
                        col_entree = f"{param}_entree"
                        col_sortie = f"{param}_sortie"
                        
                        if col_entree in merged.columns and col_sortie in merged.columns:
                            merged[f"Rendement_{param.replace('_mg_L', '')}_pct"] = (
                                (merged[col_entree] - merged[col_sortie]) / 
                                merged[col_entree] * 100
                            ).round(2)
                    
                    performance_results.append(merged)
        
        return performance_results

def main():
    """
    Fonction principale
    """
    # Vérifier que le fichier existe
    excel_file = "Excel donnés qualité UGB.xlsx"
    
    if not Path(excel_file).exists():
        print(f"❌ ERREUR : Fichier '{excel_file}' introuvable !")
        print("📁 Fichiers Excel trouvés dans le répertoire actuel :")
        excel_files = list(Path('.').glob('*.xlsx'))
        if excel_files:
            for i, file in enumerate(excel_files, 1):
                print(f"   {i}. {file.name}")
            print("\n💡 Conseil : Vérifiez le nom exact du fichier ci-dessus")
        else:
            print("   Aucun fichier .xlsx trouvé")
        return
    
    # Initialiser le processeur
    processor = UGBDataProcessor()
    
    # Traiter les données
    result = processor.create_unified_dataset(excel_file)
    
    if result is None:
        print("Échec du traitement")
        return
    
    # CORRECTION: Récupérer les résultats AVANT le diagnostic
    df_unified, performance_sheets, missing_analysis = result
    
    # Afficher un aperçu
    print("\n=== APERÇU DU DATASET FINAL ===")
    print(f"Shape: {df_unified.shape}")
    print("\nPremières lignes :")
    print(df_unified.head())
    
    print("\nColonnes disponibles :")
    for i, col in enumerate(df_unified.columns):
        print(f"{i+1:2d}. {col}")
    
    # Sauvegarder le dataset principal en CSV
    output_file_csv = "UGB_Sanar_Station_Dataset_Clean.csv"
    df_unified.to_csv(output_file_csv, index=False, encoding='utf-8')
    print(f"✓ Dataset principal sauvegardé : {output_file_csv}")
    
    # Sauvegarder l'analyse des valeurs manquantes en CSV
    missing_file_csv = "UGB_Missing_Values_Analysis.csv"
    missing_analysis.to_csv(missing_file_csv, index=False, encoding='utf-8')
    print(f"✓ Analyse des valeurs manquantes : {missing_file_csv}")
    
    # DIAGNOSTIC AVANT L'ANALYSE DE PERFORMANCE
    print("\n=== DIAGNOSTIC AVANT PERFORMANCE ===")
    print(f"Colonnes dans df_unified : {list(df_unified.columns)}")
    if 'Date' in df_unified.columns:
        print(f"Dates valides : {df_unified['Date'].notna().sum()}/{len(df_unified)}")
        
        # Vérifier les phases et filtres
        if 'Phase' in df_unified.columns:
            print(f"Phases disponibles : {df_unified['Phase'].unique()}")
        if 'ID_Filtre' in df_unified.columns:
            print(f"Filtres disponibles : {df_unified['ID_Filtre'].unique()}")
            
        # Vérifier les données par phase/filtre
        if 'Phase' in df_unified.columns and 'ID_Filtre' in df_unified.columns:
            entree_data = df_unified[df_unified['Phase'] == 'Entree']
            sortie_data = df_unified[df_unified['Phase'] == 'Sortie']
            print(f"Données d'entrée : {len(entree_data)} lignes")
            print(f"Données de sortie : {len(sortie_data)} lignes")
            
            # Détail par filtre
            for filtre_id in ['FV1', 'FV2']:
                entree_filtre = entree_data[entree_data['ID_Filtre'].isin([filtre_id, 'General'])]
                sortie_filtre = sortie_data[sortie_data['ID_Filtre'] == filtre_id]
                print(f"  - {filtre_id}: {len(entree_filtre)} entrées, {len(sortie_filtre)} sorties")
    else:
        print("❌ Colonne 'Date' manquante !")
        print("Le problème est dans le mapping des colonnes ou le traitement des feuilles.")
        return
    
    # Analyser les performances si demandé
    try:
        performance_results = processor.create_performance_analysis(df_unified, performance_sheets)
        
        if performance_results:
            df_performance = pd.concat(performance_results, ignore_index=True)
            performance_file_csv = "UGB_Performance_Analysis.csv"
            df_performance.to_csv(performance_file_csv, index=False, encoding='utf-8')
            print(f"✓ Analyse des performances : {performance_file_csv}")
        else:
            print("⚠ Aucune analyse de performance générée")
            
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse de performance : {e}")
        import traceback
        traceback.print_exc()
    
    print("\n=== TRAITEMENT TERMINÉ ===")


# Version alternative avec gestion d'erreur plus robuste
def main_with_error_handling():
    """
    Version alternative de main() avec gestion d'erreur robuste
    """
    try:
        # Vérifier que le fichier existe
        excel_file = "Excel donnés qualité UGB.xlsx"
        
        if not Path(excel_file).exists():
            print(f"❌ ERREUR : Fichier '{excel_file}' introuvable !")
            return
        
        # Initialiser le processeur
        processor = UGBDataProcessor()
        
        # Traiter les données
        print("=== DÉBUT DU TRAITEMENT ===")
        result = processor.create_unified_dataset(excel_file)
        
        if result is None:
            print("❌ Échec du traitement des données")
            return
        
        df_unified, performance_sheets, missing_analysis = result
        
        # Vérifications de sécurité
        if df_unified is None or len(df_unified) == 0:
            print("❌ Dataset vide après traitement")
            return
        
        print(f"✅ Dataset traité avec succès : {df_unified.shape}")
        
        # Diagnostic détaillé
        print("\n=== DIAGNOSTIC DÉTAILLÉ ===")
        print("Colonnes critiques :")
        critical_cols = ['Date', 'Phase', 'ID_Filtre', 'Type_Filtre']
        for col in critical_cols:
            if col in df_unified.columns:
                non_null = df_unified[col].notna().sum()
                print(f"  ✅ {col}: {non_null}/{len(df_unified)} valeurs")
                if col in ['Phase', 'ID_Filtre']:
                    print(f"      Valeurs uniques: {list(df_unified[col].dropna().unique())}")
            else:
                print(f"  ❌ {col}: MANQUANTE")
        
        # Sauvegardes
        try:
            output_file_csv = "UGB_Sanar_Station_Dataset_Clean.csv"
            df_unified.to_csv(output_file_csv, index=False, encoding='utf-8')
            print(f"\n✅ Dataset sauvegardé : {output_file_csv}")
            
            missing_file_csv = "UGB_Missing_Values_Analysis.csv"
            missing_analysis.to_csv(missing_file_csv, index=False, encoding='utf-8')
            print(f"✅ Analyse missing values : {missing_file_csv}")
            
        except Exception as e:
            print(f"❌ Erreur lors de la sauvegarde : {e}")
        
        # Analyse de performance (optionnelle)
        if 'Date' in df_unified.columns and 'Phase' in df_unified.columns:
            print("\n=== TENTATIVE D'ANALYSE DE PERFORMANCE ===")
            try:
                performance_results = processor.create_performance_analysis(df_unified, performance_sheets)
                
                if performance_results and len(performance_results) > 0:
                    df_performance = pd.concat(performance_results, ignore_index=True)
                    performance_file_csv = "UGB_Performance_Analysis.csv"
                    df_performance.to_csv(performance_file_csv, index=False, encoding='utf-8')
                    print(f"✅ Analyse de performance : {performance_file_csv}")
                else:
                    print("⚠ Aucune donnée de performance générée")
                    
            except Exception as e:
                print(f"⚠ Échec de l'analyse de performance (non critique) : {e}")
        else:
            print("⚠ Analyse de performance ignorée (colonnes manquantes)")
        
        print("\n🎉 TRAITEMENT TERMINÉ AVEC SUCCÈS")
        
    except Exception as e:
        print(f"❌ ERREUR CRITIQUE : {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    # Choisissez la version que vous préférez
    main()  # Version avec diagnostic
    # main_with_error_handling()  # Version ultra-robuste

=== DÉBUT DU PRÉTRAITEMENT UGB ===
Nombre de feuilles chargées : 13
Traitement de la feuille : Entree 1
  ✓ Entree 1: 16 lignes
Traitement de la feuille : Entree FV1
  ✓ Entree FV1: 16 lignes
Traitement de la feuille : SFV1a
  ✓ SFV1a: 16 lignes
Traitement de la feuille : SFV1b
  ✓ SFV1b: 16 lignes
Traitement de la feuille : SFV1c
  ✓ SFV1c: 16 lignes
Traitement de la feuille : Entréé FV2
  ✓ Entréé FV2: 16 lignes
Traitement de la feuille : SFV2a
  ✓ SFV2a: 16 lignes
Traitement de la feuille : SFV2b
  ✓ SFV2b: 16 lignes
Traitement de la feuille : SFHa
  ✓ SFHa: 16 lignes
Traitement de la feuille : SFHb
  ✓ SFHb: 16 lignes

Dataset unifié créé : 160 lignes, 27 colonnes

=== ANALYSE DES VALEURS MANQUANTES ===
 10 des colonnes avec le plus de valeurs manquantes :
                   Colonne  Valeurs_Manquantes  Pourcentage_Manquant
                      Mois                 160                100.00
         Heure_Echantillon                 160                100.00
      Debit_Entree_m3_

In [19]:
import pandas as pd

# Supposons que votre DataFrame nettoyé est nommé df_unified
# Si ce n'est pas déjà fait, chargez le fichier CSV dans un DataFrame
df_unified = pd.read_csv("UGB_Sanar_Station_Dataset_Clean.csv")

# Affichez le nombre de lignes et de colonnes
print(df_unified.shape)

(122, 23)
